In [101]:
import regex as re
from functional import seq
from fn import _
from collections import namedtuple
from typing import List, Dict, Tuple, Optional
from termcolor import colored
import os

In [103]:
act_dir = os.environ["HOME"]+"/tmp/nlp/ustawy"
act_file = "1993_599.txt"

In [104]:
act = open(act_dir + "/" + act_file).read()
print(act[:1000])





Dz.U. z 1993 r. Nr 129, poz. 599 
                                USTAWA
                        z dnia 9 grudnia 1993 r.
                                    
             o zmianie ustawy o podatku od towarów i usług
                        oraz o podatku akcyzowym
                                    
                                    
                                Art. 1.
W ustawie z dnia 8 stycznia 1993 r. o podatku od towarów i usług oraz o podatku
akcyzowym (Dz.U. Nr 11, poz. 50 i Nr 28, poz. 127) wprowadza się następujące
zmiany:
 1) w art. 3:
    a) dotychczasowa treść otrzymuje oznaczenie ust. 1,
    b) dodaje się ust. 2 w brzmieniu:
          "2. Rada Ministrów może określać, w drodze rozporządzenia, 
             towary i usługi, z wyjątkiem wyrobów akcyzowych i wyrobów 
             z metali szlachetnych, do których nie stosuje się przepisów 
             ustawy.";
 
 2) w art. 4:
    a) w pkt 7 wyrazy "użytkowania wynosi" zastępuje się wyrazami "używania 
      wyni

In [105]:
def match_with_index(pattern ,text:str)-> List[Tuple[int,int,str]]:
    return [(m.start(0), m.end(0),m.captures()[0]) for m in re.finditer(pattern, text)]


def print_highlighted(
    text:str,
    matches_groups:List[List[Tuple[int,int]]],
    colors:List[Tuple[str,Optional[str]]]
):
    if len(matches_groups) != len(colors):
        raise "There should be the same number of matches groups and colors"
    
    MatchedEntry=namedtuple('MatchedEntry','beg end color on_color')
    
    # Seq[MatchedEntry]
    mg = seq(matches_groups)\
        .zip(colors)\
        .flat_map(lambda gc: seq(gc[0])\
            .map(lambda x : MatchedEntry(beg=x[0],end=x[1],color=gc[1][0], on_color = gc[1][1]))
                 )\
        .order_by(_.beg)
        
    # Will duplicate some matches but that's not the point
    beg = 0
    
    for matched in mg:
        print(text[beg:matched.beg],end="") 
        print(colored(text[matched.beg:matched.end],color= matched.color, on_color= matched.on_color),end="")
        beg = matched.end

            
        
        
    
                
    

In [109]:
dzu_pat = re.compile("\Dz.U.")
year_pat=re.compile("(1|2)\d{3}")

In [113]:
years = match_with_index(year_pat,act)
dzu = match_with_index(dzu_pat,act)

print_highlighted(act,[years,dzu],[("green",None),("red","on_grey")])





Dz.U. z 1993 r. Nr 129, poz. 599 
                                USTAWA
                        z dnia 9 grudnia 1993 r.
                                    
             o zmianie ustawy o podatku od towarów i usług
                        oraz o podatku akcyzowym
                                    
                                    
                                Art. 1.
W ustawie z dnia 8 stycznia 1993 r. o podatku od towarów i usług oraz o podatku
akcyzowym (Dz.U. Nr 11, poz. 50 i Nr 28, poz. 127) wprowadza się następujące
zmiany:
 1) w art. 3:
    a) dotychczasowa treść otrzymuje oznaczenie ust. 1,
    b) dodaje się ust. 2 w brzmieniu:
          "2. Rada Ministrów może określać, w drodze rozporządzenia, 
             towary i usługi, z wyjątkiem wyrobów akcyzowych i wyrobów 
             z metali szlachetnych, do których nie stosuje się przepisów 
             ustawy.";
 
 2) w art. 4:
    a) w pkt 7 wyrazy "użytkowania wynosi" zastępuje się wyrazami "używania 
      wyni